# Kohya's Script + Lora Train `Developing`

More info: https://github.com/wibus-wee/stable_diffusion_chilloutmix_ipynb

> Created by [@wibus-wee](https://github.com/wibus-wee)
>
> Reference: [camenduru/stable-diffusion-webui-colab](https://github.com/camenduru/stable-diffusion-webui-colab)

In [ ]:
#@title 选择模型 { display-mode: "form" }

#@markdown 选择模型
checkpoint = 'chilloutmix.safetensors' #@param ["chilloutmix.safetensors", "sunshinemix.safetensors", "grapefruitHentaiModel.safetensors"]

downloadLink = {
    'chilloutmix.safetensors': 'https://civitai.com/api/download/models/11745',
    'sunshinemix.safetensors': 'https://civitai.com/api/download/models/11752',
    'grapefruitHentaiModel.safetensors': 'https://civitai.com/api/download/models/9000'
}[checkpoint]

print("已选择模型: " + checkpoint + " <===> " + downloadLink)

%store checkpoint
%store downloadLink

In [ ]:
#@title 1. 检查 GPU & 检查环境

#@markdown 此步骤将检查你的 GPU 是否支持 xformers，同时会检查是否为 Paperspace 平台（仅针对 M4000 GPU 做了判断，因此其他的 GPU 可能在此处将会出现逻辑判断错误的情况，你可能需要自行勾选）。

import os, subprocess
paperspace_m4000 = False
#@markdown 是否为 Paperspace 平台
isPaperspace = False #@param {type:"boolean"}
try:
    subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE)
    if 'M4000' in subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8'):
        print("WARNING: 你正在使用的是 Quadro M4000 GPU，它将无法使用 xformers。")
        paperspace_m4000 = True
        isPaperspace = True
    else:
        print("你正在使用的是合适的 GPU - " + subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8') + "。")
        print("平台: Paperspace" if isPaperspace else "使用平台: Colab")
except:
    print("似乎没有 GPU 可用。请检查你的运行时类型。")
    exit()

rootDir = isPaperspace and '/tmp' or '/content'
%store rootDir
%store paperspace_m4000 
%store isPaperspace

In [ ]:
#@title 2. 安装训练依赖

%store -r rootDir 
%store -r checkpoint 
%store -r downloadLink

!apt-get -y install -qq aria2
ariaInstalled = False

try:
    subprocess.run(['aria2c', '--version'], stdout=subprocess.PIPE)
    ariaInstalled = True
except:
    pass

!git clone --recurse-submodules https://github.com/Akegarasu/lora-scripts {rootDir}/lora-scripts

%cd {rootDir}/lora-scripts
if ariaInstalled:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {downloadLink} -d {rootDir}/lora-scripts/sd-models -o {checkpoint}
else:
    !wget -c {downloadLink} -P {rootDir}/stable-diffusion-webui/models/Stable-diffusion -O {rootDir}/lora-scripts/sd-models/{checkpoint}
!chmod +x install.bash
!bash install.bash

if not paperspace_m4000:
  !pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  !pip install -q --pre triton

In [ ]:
#@title 3. 训练选项设置
%store -r rootDir 
%store -r checkpoint 

!source venv/bin/activate

#@markdown 底膜路径 - 已跟随 Checkpoint 设置
pretrained_model = rootDir + "/lora-scripts/sd-models/" + checkpoint
#@markdown 训练数据集路径
train_data_dir = "./train/aki" #@param {type:"string"}
#@markdown 图片分辨率，宽,高。支持非正方形，但必须是 64 倍数。
resolution = "512,512" #@param ["256,256", "512,512", "1024,1024", "2048,2048"]
#@markdown batch size | 批大小
batch_size = 1 #@param {type:"number"}
#@markdown max train epoches | 最大训练 epoch
max_train_epoches = 10 #@param {type:"number"}
#@markdown save every n epochs | 每 N 个 epoch 保存一次
save_every_n_epochs = 2 #@param {type:"number"}
#@markdown network dim | 常用 4~128，不是越大越好
network_dim = 32 #@param {type:"number"}
#@markdown network alpha | 常用与 network_dim 相同的值或者采用较小的值，如 network_dim的一半 防止下溢。默认值为 1，使用较小的 alpha 需要提升学习率。
network_alpha = 32 #@param {type:"number"}
#@markdown clip skip | 玄学 一般用 2
clip_skip = 2 #@param {type:"number"}
#@markdown train U-Net only | 仅训练 U-Net，开启这个会牺牲效果大幅减少显存使用。6G显存可以开启
train_unet_only = False #@param {type:"boolean"}
#@markdown train Text Encoder only | 仅训练 文本编码器
train_text_encoder_only = False #@param {type:"boolean"}
#@markdown 学习率
lr = "1e-4" #@param {type:"string"}
#@markdown unet_lr | U-Net 学习率
unet_lr = "1e-4" #@param {type:"string"}
#@markdown text_encoder_lr | 文本编码器学习率
text_encoder_lr = "1e-5" #@param {type:"string"}
#@markdown lr_scheduler | "linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"
lr_scheduler = "cosine_with_restarts" #@param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"] {type:"string"}
#@markdown lr_warmup_steps | 仅在 lr_scheduler 为 constant_with_warmup 时需要填写这个值
lr_warmup_steps = 0 #@param {type:"number"}
#@markdown output model name | 模型保存名称
output_name = "aki" #@param {type:"string"}
#@markdown model save ext | 模型保存格式 ckpt, pt, safetensors
save_model_as = "safetensors" #@param ["ckpt", "pt", "safetensors"] {type:"string"}
#@markdown 若需要从已有的 LoRA 模型上继续训练，请填写 LoRA 模型路径。
network_weights = "" #@param {type:"string"}
#@markdown arb min resolution | arb 最小分辨率
min_bucket_reso = 256 #@param {type:"number"}
#@markdown arb max resolution | arb 最大分辨率
max_bucket_reso = 1024 #@param {type:"number"}
#@markdown persistent dataloader workers | 容易爆内存，保留加载训练集的worker，减少每个 epoch 之间的停顿
persistent_data_loader_workers = 0 #@param {type:"number"}
#@markdown use 8bit adam optimizer | 使用 8bit adam 优化器节省显存，默认启用。部分 10 系老显卡无法使用，修改为 0 禁用。
use_8bit_adam = True #@param {type:"boolean"}
#@markdown use lion optimizer | 使用 Lion 优化器
use_lion = False #@param {type:"boolean"}

!echo "" > _train.sh # clear
!echo "pretrained_model={pretrained_model}" >> _train.sh
!echo "train_data_dir={train_data_dir}" >> _train.sh
!echo "resolution={resolution}" >> _train.sh
!echo "batch_size={batch_size}" >> _train.sh
!echo "max_train_epoches={max_train_epoches}" >> _train.sh
!echo "save_every_n_epochs={save_every_n_epochs}" >> _train.sh
!echo "network_dim={network_dim}" >> _train.sh
!echo "network_alpha={network_alpha}" >> _train.sh
!echo "clip_skip={clip_skip}" >> _train.sh
!echo "train_unet_only={train_unet_only}" >> _train.sh
!echo "train_text_encoder_only={train_text_encoder_only}" >> _train.sh
!echo "lr={lr}" >> _train.sh
!echo "unet_lr={unet_lr}" >> _train.sh
!echo "text_encoder_lr={text_encoder_lr}" >> _train.sh
!echo "lr_scheduler={lr_scheduler}" >> _train.sh
!echo "lr_warmup_steps={lr_warmup_steps}" >> _train.sh
!echo "output_name={output_name}" >> _train.sh
!echo "save_model_as={save_model_as}" >> _train.sh
!echo "network_weights={network_weights}" >> _train.sh
!echo "min_bucket_reso={min_bucket_reso}" >> _train.sh
!echo "max_bucket_reso={max_bucket_reso}" >> _train.sh
!echo "persistent_data_loader_workers={persistent_data_loader_workers}" >> _train.sh
!echo "use_8bit_adam={use_8bit_adam}" >> _train.sh
!echo "use_lion={use_lion}" >> _train.sh

NO_NOT_MODIFY_CONTENTS = """
# ============= DO NOT MODIFY CONTENTS BELOW | 请勿修改下方内容 =====================
export HF_HOME="huggingface"
export TF_CPP_MIN_LOG_LEVEL=3
extArgs=()
if [ $train_unet_only == 1 ]; then extArgs+=("--network_train_unet_only"); fi
if [ $train_text_encoder_only == 1 ]; then extArgs+=("--network_train_text_encoder_only"); fi
if [ $network_weights ]; then extArgs+=("--network_weights $network_weights"); fi
if [ $use_8bit_adam == 1 ]; then extArgs+=("--use_8bit_adam"); fi
if [ $use_lion == 1 ]; then extArgs+=("--use_lion_optimizer"); fi
if [ $persistent_data_loader_workers == 1 ]; then extArgs+=("--persistent_data_loader_workers"); fi
accelerate launch --num_cpu_threads_per_process=8 "./sd-scripts/train_network.py" \
  --enable_bucket \
  --pretrained_model_name_or_path=$pretrained_model \
  --train_data_dir=$train_data_dir \
  --output_dir="./output" \
  --logging_dir="./logs" \
  --resolution=$resolution \
  --network_module=networks.lora \
  --max_train_epochs=$max_train_epoches \
  --learning_rate=$lr \
  --unet_lr=$unet_lr \
  --text_encoder_lr=$text_encoder_lr \
  --lr_scheduler=$lr_scheduler \
  --lr_warmup_steps=$lr_warmup_steps \
  --network_dim=$network_dim \
  --network_alpha=$network_alpha \
  --output_name=$output_name \
  --train_batch_size=$batch_size \
  --save_every_n_epochs=$save_every_n_epochs \
  --mixed_precision="fp16" \
  --save_precision="fp16" \
  --seed="1337" \
  --cache_latents \
  --clip_skip=$clip_skip \
  --prior_loss_weight=1 \
  --max_token_length=225 \
  --caption_extension=".txt" \
  --save_model_as=$save_model_as \
  --min_bucket_reso=$min_bucket_reso \
  --max_bucket_reso=$max_bucket_reso \
  --xformers --shuffle_caption ${extArgs[@]}
"""

!echo "$NO_NOT_MODIFY_CONTENTS" >> _train.sh

!echo "Finished writing _train.sh"

In [ ]:
#@title 4. 开始训练
!chmod +x _train.sh
!sh _train.sh